In [11]:
# Import libraries.
import re
import numpy as np
import tensorflow as tf
from tensorflow.contrib import learn
import os
import time
import datetime
from datetime import timedelta
from IPython.display import clear_output
import pickle

In [16]:
#function to clean the data
def clean_string(string):
    string = string.lower()
    # remove web addresses
    string = re.sub(r"(http(s)?:\/\/.)?(www\.)?[-a-zA-Z0-9@:%._\+~#=]{2,256}\.[a-z]{2,6}\b([-a-zA-Z0-9@:%_\+.~#?&//=]*)", 
                    " ", string)
    string = re.sub(r"[^a-z0-9!\?\']", " ", string)
    string = re.sub(r"'s", " 's", string)
    string = re.sub(r"s' ", " 's", string)
    string = re.sub(r"'ve", " 've", string)
    string = re.sub(r"n't", " n't", string)
    string = re.sub(r"'re", " 're", string)
    string = re.sub(r"'d", " 'd", string)
    string = re.sub(r"'ll", " 'll", string)
    string = re.sub(r"!", " ! ", string)
    string = re.sub(r"\?", " ? ", string)
    string = re.sub(r"\s{2,}", " ", string)
    return string.strip()

In [2]:
#sets values for the default graph
dropout_keep_prob = 1.0

# create batching object
with open('/share/yelp/params_voc_seq_num.pickle', 'rb') as fp:
    vocab, seq_len, num_lines = pickle.load(fp)

In [3]:
class TextCNN(object):
    """
    A CNN for text classification.
    Uses an embedding layer, followed by a convolutional, max-pooling and softmax layer.
    
    Baseline model based off of example from WildML (link below)
    http://www.wildml.com/2015/12/implementing-a-cnn-for-text-classification-in-tensorflow/
    """
    def __init__(
      self, sequence_length, num_classes, vocab_size,
      embedding_size, filter_sizes, num_filters, l2_reg_lambda=0.0):

        # Placeholders for input, output and dropout
        self.input_x = tf.placeholder(tf.int32, [None, sequence_length], name="input_x")
        self.input_y = tf.placeholder(tf.float32, [None, num_classes], name="input_y")
        self.dropout_keep_prob = tf.placeholder(tf.float32, name="dropout_keep_prob")

        # Keeping track of l2 regularization loss (optional)
        l2_loss = tf.constant(0.0)

        # Embedding layer
        with tf.device('/cpu:0'), tf.name_scope("embedding"):
            W = tf.Variable(
                tf.random_uniform([vocab_size, embedding_size], -1.0, 1.0),
                name="W")
            self.embedded_chars = tf.nn.embedding_lookup(W, self.input_x)
            self.embedded_chars_expanded = tf.expand_dims(self.embedded_chars, -1)

        # Create a convolution + maxpool layer for each filter size
        pooled_outputs = []
        for i, filter_size in enumerate(filter_sizes):
            with tf.name_scope("conv-maxpool-%s" % filter_size):
                # Convolution Layer
                filter_shape = [filter_size, embedding_size, 1, num_filters]
                W = tf.Variable(tf.truncated_normal(filter_shape, stddev=0.1), name="W")
                b = tf.Variable(tf.constant(0.1, shape=[num_filters]), name="b")
                conv = tf.nn.conv2d(
                    self.embedded_chars_expanded,
                    W,
                    strides=[1, 1, 1, 1],
                    padding="VALID",
                    name="conv")
                # Apply nonlinearity
                h = tf.nn.relu(tf.nn.bias_add(conv, b), name="relu")
                # Maxpooling over the outputs
                pooled = tf.nn.max_pool(
                    h,
                    ksize=[1, sequence_length - filter_size + 1, 1, 1],
                    strides=[1, 1, 1, 1],
                    padding='VALID',
                    name="pool")
                pooled_outputs.append(pooled)

        # Combine all the pooled features
        num_filters_total = num_filters * len(filter_sizes)
        self.h_pool = tf.concat(3, pooled_outputs)
        self.h_pool_flat = tf.reshape(self.h_pool, [-1, num_filters_total])

        # Add dropout
        with tf.name_scope("dropout"):
            self.h_drop = tf.nn.dropout(self.h_pool_flat, self.dropout_keep_prob)

        # Final (unnormalized) scores and predictions
        with tf.name_scope("output"):
            W = tf.get_variable(
                "W",
                shape=[num_filters_total, num_classes],
                initializer=tf.contrib.layers.xavier_initializer())
            b = tf.Variable(tf.constant(0.1, shape=[num_classes]), name="b")
            l2_loss += tf.nn.l2_loss(W)
            l2_loss += tf.nn.l2_loss(b)
            self.scores = tf.nn.xw_plus_b(self.h_drop, W, b, name="scores")
            self.predictions = tf.argmax(self.scores, 1, name="predictions")

        # CalculateMean cross-entropy loss
        with tf.name_scope("loss"):
            losses = tf.nn.softmax_cross_entropy_with_logits(self.scores, self.input_y)
            self.loss = tf.reduce_mean(losses) + l2_reg_lambda * l2_loss

        # Accuracy
        with tf.name_scope("accuracy"):
            correct_predictions = tf.equal(self.predictions, tf.argmax(self.input_y, 1))
            self.accuracy = tf.reduce_mean(tf.cast(correct_predictions, "float"), name="accuracy")

In [62]:
with tf.Graph().as_default():
    sess = tf.Session()
    with sess.as_default():
        cnn = TextCNN(
            sequence_length=seq_len,
            num_classes=5,
            vocab_size=len(vocab),
            embedding_size=32,
            filter_sizes=[3,4,5],
            num_filters=128)
        
        # Define Training procedure
        global_step = tf.Variable(0, name="global_step", trainable=False)
        optimizer = tf.train.AdamOptimizer(1e-3)
        grads_and_vars = optimizer.compute_gradients(cnn.loss)
        train_op = optimizer.apply_gradients(grads_and_vars, global_step=global_step)
        
        saver = tf.train.Saver(tf.all_variables())
        saver.restore(sess, '/checkpoints/model-40000')
        
        def softmax(x):
            """
            Compute softmax values for each sets of scores in x.  
            """
            return np.exp(x) / np.sum(np.exp(x), axis=-1)
        
        def predict_stars(input_str, verbose=True):
            rev = clean_string(input_str).split(' ')
            rev_arr = np.zeros((1,seq_len), dtype=int)
            for i,w in enumerate(rev):
                if not vocab.get(w):
                    rev_arr[0,i] = vocab['_RARE_']
                else:
                    rev_arr[0,i] = vocab[w]
            
            feed_dict = {cnn.input_x: rev_arr, cnn.dropout_keep_prob: 1.0}
            scores = softmax(sess.run([cnn.scores], feed_dict)).squeeze()
            if verbose:
                print('Score probabilities: {:.3f} {:.3f}, {:.3f}, {:.3f}, {:.3f}\nFor input review:\n{}'
                      .format(*scores, ' '.join(rev)))
            return scores

In [63]:
predict_stars('shitty food, terrible service')

Score probabilities: 0.883 0.102, 0.013, 0.001, 0.001
For input review:
shitty food terrible service


array([  8.82613301e-01,   1.02270745e-01,   1.29711665e-02,
         1.42480084e-03,   7.19940464e-04], dtype=float32)

In [64]:
predict_stars('the food was excellent, really had outstanding service.  order the chicken, you wont regret it')

Score probabilities: 0.002 0.005, 0.029, 0.219, 0.744
For input review:
the food was excellent really had outstanding service order the chicken you wont regret it


array([ 0.00210247,  0.00508906,  0.0291181 ,  0.21933572,  0.74435467], dtype=float32)